In [ ]:
project = 'saga-nvdb-prod-vlmh'
use_colab_auth = True

# Legg inn ditt eget prosjekt her, f.eks. 'saga-olanor-playground-ab12'
bq_job_project = ''

In [ ]:
if (use_colab_auth):
  from google.colab import auth
  auth.authenticate_user()
  print('Authenticated')

In [ ]:
import warnings
from google.cloud import bigquery

warnings.filterwarnings('ignore')
client = bigquery.Client(project=bq_job_project)

Denne spørringen henter ut ID, navn, kortform på tilhørende vegsystem, åpningsår, oppgraderingsår og geometri for tunnelløp rehabilitert siden 2020, sortert på åpningsår.

In [ ]:
query = f"""
SELECT
  id,
  egenskaper.navn,
  ARRAY(SELECT kortform FROM UNNEST(lokasjon.vegsystemreferanser)) kortform,
  egenskaper.aapningsaar,
  egenskaper.oppgraderingsaar,
  lokasjon.geometri
FROM
  `{project}.standardized.vegobjekter_tunnellop`
WHERE
  metadata.sluttdato IS NULL
  AND egenskaper.oppgraderingsaar >= 2020
ORDER BY aapningsaar
"""

print(query)

client.query(query).to_dataframe()

Denne spørringen henter ID, kortform, underlagte tunnelløp, startdato, navn og lengde for aktive undersjøiske tunnelløp, sortert på navn.

In [ ]:
query = f"""
SELECT
  id,
  (SELECT kortform FROM UNNEST(lokasjon.vegsystemreferanser)) kortform,
  (SELECT vegobjekter FROM UNNEST(relasjoner.barn) WHERE type.id = 67) tunnellop,
  metadata.startdato, 
  egenskaper.navn,
  egenskaper.lengde_offisiell
FROM
  `{project}.standardized.vegobjekter_tunnel`
WHERE
  metadata.sluttdato IS NULL
  AND egenskaper.undersjoisk = 'Ja'
ORDER BY
  navn
"""

print(query)

client.query(query).to_dataframe()